# Summary

This submission proposes then fully implements and evaluates a machine learning method for [the matching problem between career advice questions and professionals](https://www.kaggle.com/c/data-science-for-good-careervillage/overview) on [CareerVillage.org](https://www.careervillage.org/). First, we explore the data set to have a better understanding of the current scope of collected data. We then discuss how *emails* and *matches* generated by [the current tag-based recommendation system](https://www.kaggle.com/c/data-science-for-good-careervillage/discussion/84845#latest-510508) together with *questions* and *answers* data tables  can be used to build a supervised machine learning (ML) data set for the matching problem. Next, four feature engineering steps are implemented to demonstrate how different types of data including activities, network interactions, and texts can be used to improve matching predictions. Two machine learning models, conditional logistic regression and gradient boosting decision trees, are then trained and compared on a test set from July 2018 to Jan 2019. The performance comparison results demonstrate that our proposed solution can help to improve the accuracy of the current tag-based recommendation approach and significantly cut down the number of recommendations sent to professionals by 15 times. The released recommendation volume, therefore, can be used to send more questions to professionals that do not register for the same tags with new questions but still highly ranked by the models. A/B experiments using this new recommendation approach in turn will help us collect more training data and improve our supervised ML model further. Other extensions to improve the proposed approach are also discussed in the future work section. To demonstrate the feasibility of these potential extensions, we implements a supervised deep learning model that can be used to capture latent semantics between tags, effectively allowing us to move beyond the current exact tag matching recommendation approach.

# Notes #
Our submission includes this report and a sequence of other kernels where Python code, output data, and detailed discussions can be found. There are two reasons to divide the work into multiple notebooks:
* Modularization allows us to develop, revise, and test each data processing and modelling step independently. For example, we can start a new modelling step while waiting for the finish of the previous step's kernel. 
* Some steps such as text processing and cross-validation grid search can take long time to run. An undivided notebook will exceed the kernel run time limit.

# Table of Contents and Notebooks 

This [network and nlp modelling for careervillage notebook](https://www.kaggle.com/titatum/network-and-nlp-modelling-for-careervillage) summarizes our approach to the career advice question recommendation problem and provide links to other notebooks where code, data, and visualization outputs are listed and discussed in more details.

**I. Exploratory Data Analysis**

The [exploratory data analysis notebook](https://www.kaggle.com/titatum/cv-exploratory-data-analysis) explores the data set provided by [CareerVillage.org](https://www.careervillage.org/) which in turn helps us to formulate the question recommendation problem as a supervised machine learning problem.

**II. Supervised Machine Learning for Question Recommendation**

The [data construction notebook](https://www.kaggle.com/titatum/cv-machine-learning-data-construction) demonstrates how *emails* and *matches* tables generated by the current recommendation system can be combined with *questions* and *answers* tables to create a data set for our supervised machine learning approach.

**III. Feature Engineering**

We consider 4 groups of predictive features based on activity, network, and text data. Each group of features is generated by one notebook and augmented to the supervised machine learning data set by another notebook.

*III.1. Activity Predictors*

These [activity predictors](https://www.kaggle.com/titatum/cv-feature-engineering-activity-predictors) and [activity predictors data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-activity-predictors) notebooks generate activity-based features such as *time since the last activity* or *the number of activities within 30 days*.

*III.2. Static Network Predictors*

These [static network predictors](https://www.kaggle.com/titatum/cv-feature-engineering-network-predictors-1) and [static network predictors data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-network-predictors-1) notebooks generate static network features such as *the number of shared tags between the questioner and the professional*.

*III.3. Dynamic Network Predictors*

These [dynamic network predictors](https://www.kaggle.com/titatum/cv-feature-engineering-network-predictors-2) and [dynamic network predictors data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-network-predictors-2) notebooks generate dynamic network features such as *the number of previous Q&A interactions between the questioner and the professional*.

*III.4. Latent Semantic Indexing (LSI) Predictors*

These [LSI scores](https://www.kaggle.com/titatum/cv-feature-engineering-text-scores) and [LSI scores data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-text-scores) notebooks generate LSI matching scores between the question and documents (i.e. questions, answers, and comments) that the professional has produced or engaged.

**IV. Machine Learning Models** 

*IV.1. Conditional Logistic Regression (CLR)*

The [conditional logistic regression notebook](https://www.kaggle.com/titatum/cv-machine-learning-conditional-lr) demonstrates how a CLR model can be used to produce matching scores between the question and potential professionals based on predictive features engineered above.

*IV.2. Gradient Boosting Decision Trees (GBDTs)*

The [gradient boosting decision trees notebook](https://www.kaggle.com/titatum/cv-machine-learning-gbdts) demonstrates how a GBDTs model can be used to produce matching scores between the question and potential professionals based on predictive features engineered above.

*IV.3. Performance Evaluation *

The [machine learning comparison notebook](https://www.kaggle.com/titatum/cv-machine-learning-comparison) compares the performance of two machine learning models on a test set from July 2018 to January 2019.

**V. Conclusions and Future Work**

The [tags embedding notebook](https://www.kaggle.com/titatum/cv-tag-embeddings) demonstrates how a deep learning model can be used to capture latent semantics between tags therefore allows us to move beyond the current exact tag matching recommendation approach, i.e. recommending questions to professionals that do not share the same tags.

**References**

**Acknowledgement**

In [ ]:
import os 
from IPython.display import Image
import pandas as pd
from scipy import stats

input_dir = '../input'
eda_dir = os.path.join(input_dir,'cv-exploratory-data-analysis')
ml_gbdts_dir = os.path.join(input_dir,'cv-machine-learning-gbdts')
ml_comp_dir = os.path.join(input_dir,'cv-machine-learning-comparison')
ml_data_dir = os.path.join(input_dir,'cv-data-augmentation-text-scores')

# I. Exploratory Data Analysis #

Full exploratory data analysis results can be found at our [EDA notebook](https://www.kaggle.com/titatum/cv-exploratory-data-analysis). In this section, we will only summarize some main observations that are relevant to our formulation of the question recommendation problem as a supervised machine learning problem.

## I.1. A healthy growth in active professionals, students, and career advice questions

CareerVillage.org is enjoying a healthy growth in both professionals and students with their career advice questions. The acceleration period has started from 2016. Three plots below show the cumulative counts of professionals and students joining the website as well as questions created over time.

In [ ]:
Image(os.path.join(eda_dir, 'CumulativeCountofProfessionalsoverTime.jpg'))

In [ ]:
Image(os.path.join(eda_dir, 'CumulativeCountofStudentsoverTime.jpg'))

In [ ]:
Image(os.path.join(eda_dir, 'CumulativeCountofQuestionsoverTime.jpg'))

The below plot shows the monthly rolling numbers of active professionals who make at least one question, answer, or comment contribution to the website in a month. Recently, at any time we observe more than 200 professionals helping students with their career advice questions.

In [ ]:
Image(os.path.join(eda_dir, '30-daywindowedActiveProfessionalsoverTime.jpg'))

The below plot shows the histogram of the numbers of answers per question. Each question that was responsed received two answers on average. However, the distribution is also skewed. Most questions receive one or two answers while some questions have more than 10 answers. 

In [ ]:
Image(os.path.join(eda_dir, 'HistogramofQuestionsbyAnswerNumbers.jpg'))

## I.2. The current tag-based recommendation system

The current tag-based recommendation system sends questions to professionals through emails. Together with the question growth, the number of emails also increases quickly over time as shown in the plot below. As questions keep increasing, the professionals can be overwhelmed by the email requests. Therefore, building an effective recommendation engine is very important to reduce the numbers of recommendations and emails sent to professionals and keep them engaged.

In [ ]:
Image(os.path.join(eda_dir, 'CumulativeCountofEmailsoverTime.jpg'))

The below plot shows the histogram of the number of recommendation emails per question. On average, a large number of emails (nearly 200) is sent to professionals on each career advice question.

In [ ]:
Image(os.path.join(eda_dir, 'HistogramofQuestionsbyEmailNumbers.jpg'))

Professionals could also be overwhelmed by a large number of recommendations per active day. In an active day when a professional receives some recommendation emails, she or he could receive up to 7 questions on average. The below plot shows the histogram of the number of recommendations per professional in an active day.

In [ ]:
Image(os.path.join(eda_dir, 'questions_in_emails_per professional_in_an_active_day.jpg'))

The main problem with the current tag-based recommendation system is the sparsity of tag information. Each question on average has 3 tags while most users also register for a very small number of tags. The below plots show the histograms of the numbers of tags per user and per question, respectively.

In [ ]:
Image(os.path.join(eda_dir, 'HistogramofUsersbyTagNumbers.jpg'))

In [ ]:
Image(os.path.join(eda_dir, 'HistogramofQuestionsbyTagNumbers.jpg'))

** Fortunately, data collected by the current tag-based recommendation system can help us to apply a supervised machine learning (ML) approach to the problem. In next sections, we will demonstrate that under our proposed ML approach, other data including activities, network interactions, and texts can be leveraged to improve the current system further. **

# II. Supervised Machine Learning for Question Recommendation #


** The current question matching problem can be considered as a binary classification problem in machine learning. **

** For each question: ** 
* ** Each of its answers in the database can be used as a matched instance. Its class label is 1. **
* ** Any recommendation sent by the system that did not result in an answer can be used as an unmatched instance. Its class label is 0. **

Under this supervised ML approach, professionals will be scored and ranked for each new question. Recommendations are only sent to highest ranked professionals. The selection of optimal cut points will be discussed in the performance evalution section. Below are some advantages of considering our question matching problem under the binary classification perspective:
* Although, we will mainly focus on predicting professionals given a question in later discussions, the same model can be used to predict questions given a professional. In particular, after training the model, we can use it to score and rank all professionals given the question or all questions given the professional. 
* Through feature engineering, the binary classification approach also allows us to partially solve the *cold start* problem for professionals who just signed up but have not yet answered any question. We will demonstrate how activity predictors such as *days from joined dates* can handle this problem.
* Potential matches for new questions which do not share tags with any professional can still be computed thanks to predictors extracted from other data of these questions such as text matching scores or previous interactions between students who ask the questions and the potential professionals.

The [data construction notebook](https://www.kaggle.com/titatum/cv-machine-learning-data-construction) contains code to generate this supervised ML data set using four data tables: *emails, matches, questions, and answers*. The resulted ML data set is very skewed with a small number of matched instances, i.e. 0.5% of the data set.

In [ ]:
examples = pd.read_parquet(os.path.join(ml_data_dir,'positive_negative_examples.parquet.gzip'))
print('The total number of instances in the ML data set is {}'.format(examples.shape[0]))
print('The total number of matched instances in the ML data set is {}'.format(examples[examples['matched']==1].shape[0]))
print('The total number of unmatched instances in the ML data set is {}'.format(examples[examples['matched']==0].shape[0]))
print('The percentage of matched instances in the ML data set is {}%'.format(
    round(100 * examples[examples['matched']==1].shape[0] / examples[examples['matched']==0].shape[0], 2)))

# III. Feature Engineering #

In this submission, we consider four groups of predictive features: activity statistics, static and dynamic network statistics, and text matching scores. Through a sequence of feature engineering steps, we would like to demonstrate that many data tables already collected in the current database can be used to improve the performance of the current tag-based recommendation system. The pipelined approach for feature construction and data augmentation also shows the extensibility of our proposed solution. More predictive features based on old or new data tables can be developed, generated, and augmented independently from each other. 

From the network or graph analytics perspective, we can consider students, professionals even their questions, answers, comments as nodes and interactions between these objects are edges [[1]](https://neo4j.com/graph-algorithms-book/). At any time point when matching decisions need to be made, we can compute different network statistics around a professional to measure her or his probability to answer new questions. Another introduction to our current approach on feature engineering with activity data, network interactions, and text matching scores can be found in the following paper [[2]](https://www.sciencedirect.com/science/article/pii/S0378873315000477). In this submission, all static and dynamic network statistics are computed using Python code. In production, a full graph analytics database can be used instead to ease the development of network predictors.

Note that in this section, the word *feature* is used interchangeably with *predictor* and *statistic*.

## III.1. Activity Predictors

These [activity predictors](https://www.kaggle.com/titatum/cv-feature-engineering-activity-predictors) and [activity predictors data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-activity-predictors) notebooks generate activity-based features. Below are definitions and intepretation of these activity features.
* **Days from joined dates**: how many days from the date when the professional joined the website to the current date when we need to make matching decisions. It helps to take into account the tenure of professionals in predicting matching scores. Note that this predictor is available even for professionals who just signed up but have not yet answered any question which in turn partially helps to solve the *cold start* problem.
* **Days from last activities**: how many days from the date when the professional made the last contribution to the website (i.e. questioning, answering, or commenting) to the current date when we need to make matching decisions. It helps to take into account the recency effect, i.e. those professionals who recently make contributions to the website are more likely to help answer new questions. 
* **Numbers of activities within 30, 365, or 100000 days**: how many activities (i.e. questioning, answering, or commenting) that a professional have made in the last 30 days, 365 days, or from the date she or he joined the website. We expect that highly active professionals in the past are more likely to contribute in the future. Note that all activities  not only answering ones are counted in these statistics; therefore, in future when clickstream data included, view activities can also be added. The above *cold start* problem can be reduced further.

Two mean difference tests below show that the values of **days from last activities** and **numbers of activities within 30 days** of matched instances are very different from unmatched instances.   

In [ ]:
unmatched_data = examples[examples['matched']==0]
matched_data = examples[examples['matched']==1]

print('The p-value of the test for the difference in \'days from last activities\' means of matched and unmatched recommendations is {}.'.format(
    stats.ttest_ind(matched_data['days_from_last_activities'], unmatched_data['days_from_last_activities'], equal_var = False)[1]))

print('The p-value of the test for the difference in \'numbers of activities within 30 days\' means of matched and unmatched recommendations is {}.'.format(
    stats.ttest_ind(matched_data['professional_activities_sum_30'], unmatched_data['professional_activities_sum_30'], equal_var = False)[1]))

## III.2. Static Network Predictors

These [static network predictors](https://www.kaggle.com/titatum/cv-feature-engineering-network-predictors-1) and [static network predictors data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-network-predictors-1) notebooks generate static network features. Below are definitions and intepretation of these static network predictors.

* **Questioner-answerer shared tags**: the number of shared tags between the student who asks the question and the professional who is being considered for sending a recommendation. The hypothesis is if a professional who is *sharing the same tags* with the student who is asking the question is more likely to address it. 

* **Questioner-answerer shared schools**: the number of shared schools between the student who asks the question and the professional who is being considered for sending a recommendation. The hypothesis is if a professional who is *sharing the same schools* with the student who is asking the question is more likely to address it. 

* **Questioner-answerer shared groups**: the number of shared groups between the student who asks the question and the professional who is being considered for sending a recommendation. The hypothesis is if a professional who is *sharing the same groups* with the student who is asking the question is more likely to address it. 

The mean difference test below shows that the values of **questioner-answerer shared tags** of matched instances are very different from unmatched instances.   

In [ ]:
print('The p-value of the test for the difference in \'questioner_answerer_shared_tags\' means of matched and unmatched recommendations is {}.'.format(
    stats.ttest_ind(matched_data['questioner_answerer_shared_tags'], unmatched_data['questioner_answerer_shared_tags'], equal_var = False)[1]))

** If timestamps are added to user tags, school, and group memberships data tables, these network predictors can be computed as dynamic ones as demonstrated in the next section. **

## III.3. Dynamic Network Predictors

These [dynamic network predictors](https://www.kaggle.com/titatum/cv-feature-engineering-network-predictors-2) and [dynamic network predictors data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-network-predictors-2) notebooks generate dynamic network features. Below are definitions and intepretation of these dynamic network predictors.

* **Questioner-answerer paths**: the number of previous questioning & answering (Q&A) interactions between the student who asks the current question and the professional who is being considered for sending a recommendation. If they have Q&A interactions with each other in the past, they are more likely to repeat their engagement in the future. Since these future interactions if happen will create circles in the Q&A graph, this effect is also called [network closure](https://en.wikipedia.org/wiki/Triadic_closure).

* **Commenter-questioner paths**: the number of previous commenting & questioning (C&Q) interactions between the student who asks the current question and the professional who is being considered for sending a recommendation. If they have C&Q interactions with each other in the past, they are more likely to repeat their engagement in the future. 

* **Commenter-answerer paths**: the number of previous commenting & answering (C&A) interactions between the student who asks the current question and the professional who is being considered for sending a recommendation. If they have C&A interactions with each other in the past, they are more likely to repeat their engagement in the future. 

The mean difference test below shows that the values of **questioner-answerer paths** of matched instances are very different from unmatched instances.   

In [ ]:
print('The p-value of the test for the difference in \'questioner_answerer_shared_tags\' means of matched and unmatched recommendations is {}.'.format(
    stats.ttest_ind(matched_data['questioners_answerers_paths'], unmatched_data['questioners_answerers_paths'], equal_var = False)[1]))

## III.4. Latent Semantic Indexing (LSI) Predictors

These [LSI scores](https://www.kaggle.com/titatum/cv-feature-engineering-text-scores) and [LSI scores data augmentation](https://www.kaggle.com/titatum/cv-data-augmentation-text-scores) notebooks generate LSI matching scores between the current question and documents (i.e. questions, answers, and comments) that the professional has directly produced or engaged.

**LSI Score**: the dot product between the LSI vector of the question and the LSI vector of the professional who is being considered for sending a recommendation. 

Before the calculation of the LSI matching score for every instance in the data set, the following steps need to be carried:

* Estimating a [latent semantic indexing](https://en.wikipedia.org/wiki/Latent_semantic_analysis) model based on all texts including questions, answers, and comments produced by professionals,

* Computing the LSI vector for each question using the above LSI model,

* Computing the LSI vector for each professional using all texts that she or he has directly produced or indirectly engaged.

The mean difference test below shows that the values of **LSI Score** of matched instances are very different from unmatched instances.   

In [ ]:
print('The p-value of the test for the difference in \'questioner_answerer_shared_tags\' means of matched and unmatched recommendations is {}.'.format(
    stats.ttest_ind(matched_data['LSI_Score'], unmatched_data['LSI_Score'], equal_var = False)[1]))

There are many other possibilities to compute text matching scores in the unsupervised manner such as TF-IDF and latent dirichlet allocation or the supervised manner such as deep learning models using word embeddings. They will be discussed in more details in the future work section.

# IV. Machine Learning Models #

After the feature engineering step, the supervised ML data set is divided into two train and test sets. The test set is from July 1, 2018 to January, 31 2019. The first part of the [gradient boosting decision trees notebook](https://www.kaggle.com/titatum/cv-machine-learning-gbdts) carries out the preparation of train and test sets. 

Below are some statistics on these train and test sets:
* The numbers of instances in train and test data sets are 9,669,240 and 730,005, respectively. 
* In the train data set, there are 9,622,427 unmatched and 4,6813 matched instances.
* In the test data set, there are 725,628 unmatched and 4,377 matched instances.


In this submission, we focus on two machine learning models: 
* Conditional logistic regression is our base line for comparison,
* Gradient boosting decion trees (GBDTs) is our proposed solution.

We chose GBDTs, a highly successful ML algorithm for many practical data science problems, for this submission since it can model nonlinear effects of a possibily large number of predictors. This allows us to easily add more complex features in future iterations to improve our matching prediction. We will discuss, in the future work section, potential directions of using deep learning models to improve our current solution further.

## IV.1. Conditional Logistic Regression

The [conditional logistic regression  (CLR) model](https://en.wikipedia.org/wiki/Conditional_logistic_regression) which allows us to stratify both matched and unmatched recommendations of a question into a group when computing the maximum likelihood estimation will serve as our simple baseline to compare with our main gradient boosting decision trees. The [conditional logistic regression notebook](https://www.kaggle.com/titatum/cv-machine-learning-conditional-lr) in R demonstrates how a CLR model can be used to produce matching scores between the question and potential professionals based on predictive features engineered above.

## IV.2. Gradient Boosting Decision Trees

The [gradient boosting decision trees for binary classification](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) is our main model. It allows us to model nonlinear effects of our predictors on matching probabilities. Code and output files of predictied matching scores and visualization plots from our GBDTs model can be found in the [gradient boosting decision trees notebook](https://www.kaggle.com/titatum/cv-machine-learning-gbdts).

### IV.2.1 Feature Importance

The below plot shows the importance of all features in the proposed GBDTs model. All four groups of features contribute at least one significant predictor to the model.
* **Activity predictors**: all 5 activity predictors are important. Both **days from joined dates** and **days from last activities** are very good predictors for matching scores.
* **Static network predictors**: only **questioner-answerer shared tags** predictor contributes to the model, but it is the weakest one.
* **Dynamic network predictors**: only **questioner-answerer paths** predictor contributes to the model, but it is the strongest one.
* **LSI predictors**: the single **LSI score** predictor contributes significantly to the model.

**These effects support our hypothesis that many data tables already collected in the current database can be used to boost the performance of the current tag-based recommendation system.**

In [ ]:
Image(os.path.join(ml_gbdts_dir, 'feature_importance.jpg'))

### IV.2.2 Feature Interpretation

We will select some representative predictors from four feature groups to interpret their effects on predicting matching probabilities. With gradient boosting decision trees, the below [partial dependence plots](https://scikit-learn.org/stable/auto_examples/ensemble/plot_partial_dependence.html) are very helpful in interpreting feature effects. On these plots, the x axis corresponds to different values of the predictor under consideration. The y axis shows the matching probabilities at different values of the predictor.

**Days from joined dates**

Conditioning on all other predictors, we observe that professionals who have recently joined the website within a year are more likely to answer new questions than those who joined it from 1 to 3 years ago. There are also some long-term contributors who joined the website more than 3 years ago and is very active in answering.

In [ ]:
Image(os.path.join(ml_gbdts_dir, 'days_from_joined_dates.jpg'))

**Days from last activities**

The smaller value of *days from last activities*, the higher matching probability. It confirms the recency hypothesis that recently active professionals are more likely to response to a question.

In [ ]:
Image(os.path.join(ml_gbdts_dir, 'days_from_last_activities.jpg'))

** Questioner-answerer shared tags **

The smaller value of *questioner-answerer shared tags*, the higher matching probability. This effect seems to counter the basis of the current tag-based recommendation. Tag data are sparse and our recommendation system needs to leverage other collected data tables.

In [ ]:
Image(os.path.join(ml_gbdts_dir, 'questioner_answerer_shared_tags.jpg'))

** Questioner-answerer paths **

The higher value of *questioner-answerer paths*, the higher matching probability. This result is a strong evidence for the network closure effect where students and professionals who interact in the past are more likely to engage with each other in the future.

In [ ]:
Image(os.path.join(ml_gbdts_dir, 'questioners_answerers_paths.jpg'))

** LSI score **

The higher value of *LSI score*, the higher matching probability. This confirms that texts that professionals have directly produced or indirectly engaged can be used to match against texts of new questions.

In [ ]:
Image(os.path.join(ml_gbdts_dir, 'LSI_Score.jpg'))

## IV.3. Performance Evaluation

In the [machine learning comparison notebook](https://www.kaggle.com/titatum/cv-machine-learning-comparison), we first compare the performance of two machine learning models, CLR and GBDTs, on a test set from July 2018 to January 2019. We then discuss some potential implications of the improvement made by our proposed model on the current tag-based system.

### IV.3.1 Performance Comparison

We compare the prediction performance of two ML models based on recall metrics. For each test question, we will compute the scores and ranks of all test instances then check if a model can retrieve at least one or all matched instances in its top K list. We vary K from 1 to 20. 
* We define **at least one recall** as the proportion of test questions that the top K list of a model succeeds to retrieve at least one matched instance.
* We define **full recall** as the proportion of test questions that the top K list of a model succeeds to retrieve all matched instance.

The two plots below show the performance of CLR and GBDTs models on two at least one and full recall metrics. On both metrics, the GBDTs model outperforms the CLR one. For example, using the list of top 10 recommendations, the GBDTs model can retrieve at least one match for 96% of test questions while the corresponding rate of the CLR model is 89%. The rates for retrieving all matched instances are 84% for GBDTs and 74% for CLR, respectively. Probably, the nonlinear effects that GBDTs can model helps it predict matching probabilities better. Therefore, we will focus our next discussion on the GBDTs model. 

In [ ]:
Image(os.path.join(ml_comp_dir,'min_rank_recommendation_recall.jpg'))

In [ ]:
Image(os.path.join(ml_comp_dir,'max_rank_recommendation_recall.jpg'))

### IV.3.2 Performance Improvement Implications

Below are some performance statistics on the test set using the top 20 recommendations from our GBDTs model:
* The total number of correct test recommendations: 4,377
* The total number of correct ML top-20 recommendations: 3,968
* The ML top-20 recall is 90.7%
* The total number of original recommendations: 730,005
* The total number of ML top-20 recommendations: 47,189
* The decrease in the number of sent recommendations is 15.5 folds

**Using the top 20 recommendations from our GBDTs model, we can still maintain a high recall but the volume of recommendations to send out can be reduced significantly by more than 15 times, compared to the current tag-based method.** Note that the recall of the tag-based method is much lower than 100%. Many answers in the test set can not be joined with items in the *matches* table. If these answers are the results of direct interactions between professionals and the website, then the recall of the current tag-based method is only around 32%, i.e. 1,395 out of 4,377 anwers in the test set. Though we are not confident about this assumption which must be confirmed by CareerVillage, we believe that the 90.7% recall of the top-20 recommendations from the GBDTs model is highly competitive. 

** Moreover, on average the number of recommendations that each professional receives in emails is also reduced by half from 3.6 to 1.8. ** Two plots below show the distributions of the numbers of questions that professionals receive in emails in the test set before (i.e. tag-based only) and after using the proposed ML GBDTs model (i.e. filtering tag-based recommendations further by the top 20 ML lists).

In [ ]:
Image(os.path.join(ml_comp_dir,'before_ml_questions_in_each_email.jpg'))

In [ ]:
Image(os.path.join(ml_comp_dir,'after_ml_questions_in_each_email.jpg'))

In [ ]:
import datetime as dt
test_examples = examples[examples['questions_date_added']>=dt.datetime(2018,7,1)]
print('The number of answers in the test set that correspond to an item in the matches table is {}.'.format(
    test_examples[((test_examples['matched']==1) & (test_examples['questions_date_added']!=test_examples['emails_date_sent']))].shape[0]))

# V. Conclusions and Future Work #

In this submission, we have proposed and demonstrated the superior performance of our supervised machine learning for the career advice Q&A matching problem compared to the current tag-based recommendation system. Using the top 20 recommendations from our GBDTs model, we can maintain a high recall but significantly reduce the volume of recommendations to send out by more than 15 times. The number of recommendations that each professional receives in emails is also reduced by half. Although we have mainly focused on predicting professionals given a question, the same model can be used to predict questions given a professional.

Regarding to the data analytics and modelling aspects, there are many potential venues for the future work. We group them into three main groups: A/B experiments for supervised learning data collection, feature engineering iterations, and deep learning models.

### V.1. A/B Experiments for Supervised Learning Data Collection

As demonstrated in the performance evaluation section, the recommendation volume of the current tag-based system can be reduced by 15 times without compromising the recall. The freed recommendation volume then can be used to send new questions to professionals that do not share the same tags with the questions but are still highly ranked by the model. Together with well designed A/B experiments this approach can help us collect more training data to improve our supervised ML models further. For example, we can send each question to professionals by:
* Group 1: retaining only 20 recommedations from the current tag-based system that are also ranked in the top 20 by the GBDTs model,
* Group 2: sending the question to additional 20 professionals who are not recommended by the tag-based system but are ranked highest by the GBDTs model,
* Group 3: sending the question to additional 20 random professionals who have not been selected yet.

The sample size of each above group should be computed depending on the desired statistical power to detect the differences between Group 1 versus Group 3 and Group 2 versus Group 3.

Data collected by these A/B experiments in turn can be added to the current supervised ML data set to train new models. In production, models also need to be updated in a rolling fashion, i.e. being retrained weekly or monthly.

### V.2. Feature Engineering Iterations

The feature engineering steps in this submission by all means are not exhaustive. More feature engineering iterations and experiments should be carried out to add more relevant features to improve the model prediction. 

* More features can be created with the current data tables including time-weighted activity counts or more complex network predictors as suggested in the following paper [[2]](https://www.sciencedirect.com/science/article/pii/S0378873315000477), or matching predictors between students and professionals' locations at the country level, ...

* New features can also be computed from newly added data tables such as *Hearts* scores for questions and answers. A potential predictor could be the average heart score that a professional has received over all her or his answers. Is it true that a highly rated professional is more eager to help answer new questions?

* New data tables can also be collected to help us create more predictive features. For example, since many answers potentially are the results of direct interactions between professionals and the website. We can collect clickstream data to create new features that can distinguish these highly engaged professionals from others.

### V.3. Deep Learning Models

The GBDTs model has shown its superior performance compared to the current baseline tag-based system on the test set. If A/B experiments confirm these results and the GBDTs model can be successfully put into production, then the GBDTs model itself will becomes a new baseline to be refined. We believe that carefully designed deep learning models can deliver further improvements thanks to the richness of our collected data set.

* **Entity embeddings** can be used to learn a lower dimension of locations, allowing us to compute match scores between students and professionals' locations. [The following Kaggle notebook](https://www.kaggle.com/aquatic/entity-embedding-neural-net) by [Joe Eddy](https://www.kaggle.com/aquatic) demonstrates how these entity embeddings can be learned uinsg Keras deep learning library.

* **Tag and text embeddings** can be learned using supervised deep learning models to capture more discriminative latent semantics of tags and texts on predicting matching probabilities. Note that the latent semantic indexing approach in this submission is unsupervised, and hence expected to have a lower predictive power than those trained under the supervised manner. In [the following tag embeddings notebook](https://www.kaggle.com/titatum/cv-tag-embeddings) we implement a deep learning model to demonstrate how tag embeddings can be learned from our current supervised ML data set. Latent tag semantics vectors, rather than the exact tags, can then be used to match questions and professionals. The preliminary experiment results are promising. Using only tag data registered by questions and professionals, we can still predict the *matched* and *unmatched* labels for test questions at the accuracy of 76%. By combining this method with more data as discussed above, we believe that a higher recommendation performance can be delivered.

* **Graph neutral convolutions** [[3]](https://arxiv.org/abs/1703.06103) can also be explored to ease the feature engineering process so that the highly complex interaction graph between students and professionals can be fully exploited.

# References

[[1] Needham M, Hodler EA. O'Reilly Media. Graph Algorithms: Practical Examples in Apache Spark and Neo4j](https://neo4j.com/graph-algorithms-book/)

[[2] Vu DQ, Pattison P, Robins G (2015). Relational Event Models for Social Learning in MOOCs. Social Networks, Volume 43, 121 - 135.](https://www.sciencedirect.com/science/article/pii/S0378873315000477)

[[3] M. Schlichtkrull, T. N. Kipf, P. Bloem, R. vd Berg, I. Titov, M. Welling, Modeling Relational Data with Graph Convolutional Networks, (ESWC 2018).](https://arxiv.org/abs/1703.06103)

# Acknowledgement

We would like to thank CareerVillage.org for the great opportunity to work with a rich and interesting data set. We also would like to thank [Daniel Becker](https://www.kaggle.com/danielbecker/careervillage-org-recommendation-engine) for some of his Python text processing code.